# Analysis

Checking the impact of drafts for a team's success.  
The only success factor worth measuring here is the stanley cup final (because that was the most accessible data)  
I have the winner and runner-up year over year for multiple years.  

### Steps  

What this will do is, for each season, it will get the teams in the final.  
It will determine their ID using the Franchises table.  
It will take this ID to the player seasons table, and create a roster for the given season.  
We will point this roster back at the drafts table, and detemrine how many of those players were drafted by the franchise.  
This gives us some insight into how impactful draft picks are on a team's success vs. non-drafted players (i.e. trade, FA).  

In [92]:
import pymysql.cursors
import config as cfg
import pandas as pd

In [49]:
db_conn = pymysql.connect(host=cfg.host,
                              user=cfg.user,
                              password=cfg.pw,
                              db=cfg.db,
                              charset='utf8mb4',
                              cursorclass=pymysql.cursors.DictCursor)

In [18]:
query1 = "SELECT * FROM nhldrafts.playofffinals;"
cur = db_conn.cursor()
cur.execute(query1)
db_conn.commit()
finals_list = cur.fetchall()

In [19]:
print(finals_list)

[{'FinalsID': 1, 'Season': 2020, 'Champion': 17, 'RunnerUp': 0}, {'FinalsID': 2, 'Season': 2019, 'Champion': 25, 'RunnerUp': 28}, {'FinalsID': 3, 'Season': 2018, 'Champion': 8, 'RunnerUp': 31}, {'FinalsID': 4, 'Season': 2017, 'Champion': 9, 'RunnerUp': 14}, {'FinalsID': 5, 'Season': 2016, 'Champion': 9, 'RunnerUp': 18}, {'FinalsID': 6, 'Season': 2015, 'Champion': 7, 'RunnerUp': 17}, {'FinalsID': 7, 'Season': 2014, 'Champion': 2, 'RunnerUp': 22}, {'FinalsID': 8, 'Season': 2013, 'Champion': 7, 'RunnerUp': 28}, {'FinalsID': 9, 'Season': 2012, 'Champion': 2, 'RunnerUp': 11}, {'FinalsID': 10, 'Season': 2011, 'Champion': 28, 'RunnerUp': 6}, {'FinalsID': 11, 'Season': 2010, 'Champion': 7, 'RunnerUp': 16}, {'FinalsID': 12, 'Season': 2009, 'Champion': 9, 'RunnerUp': 27}, {'FinalsID': 13, 'Season': 2008, 'Champion': 27, 'RunnerUp': 9}, {'FinalsID': 14, 'Season': 2007, 'Champion': 12, 'RunnerUp': 29}, {'FinalsID': 15, 'Season': 2006, 'Champion': 24, 'RunnerUp': 13}, {'FinalsID': 16, 'Season': 200

In [61]:
# final query
final_team_query = "SELECT * FROM nhldrafts.Franchises WHERE FranchiseID = %s OR FranchiseID = %s;"

# players query
players_season_query = "SELECT * FROM nhldrafts.PlayerSeason WHERE"

# draft query
draft_query = "SELECT * FROM nhldrafts.Drafts WHERE Player = %s;"

In [ ]:
# get teams in final
# get rosters
# get draft pick

# but
# we can do that in this order
# get teams
# for each team in final, get players on roster
# for each player on roster, get draft info
# so: team - players group - add draft stat to players group


In [80]:
finals = []
for final in finals_list[3:-1]:
    season = final['Season']
    final_teams = {}
    final_teams['Season'] = season
    final_teams['ChampionID'] = final['Champion']
    final_teams['RunnerUpID'] = final['RunnerUp']
    cur = db_conn.cursor()
    cur.execute(final_team_query, (winner, loser))
    db_conn.commit()
    _teams = cur.fetchall()
    final_teams['ChampionName'] = _teams[0]['TeamName']
    final_teams['RunnerUpName'] = _teams[1]['TeamName']
    final_teams['ChampionRoster'] = []
    final_teams['RunnerUpRoster'] = []
    
    final_teams['ChampionCounts'] = {'Drafted':0, 'NonDrafted':0}
    final_teams['RunnerUpCounts'] = {'Drafted':0, 'NonDrafted':0}
    
    # for the two teams in final_teams
    for k, v in final_teams.items():
        if k not in ['ChampionID', 'RunnerUpID'] :
            continue
        # key is Championid or RunnerUpid, v is teamid
        # get the roster of current team
        cur = db_conn.cursor()
        players_season_query_u = "{} Season = {} AND Team = {};".format(players_season_query, season, v)
        cur.execute(players_season_query_u)
        db_conn.commit()
        roster = cur.fetchall()
        # for each player in the roster, get their draft team
        for player in roster:
            cur = db_conn.cursor()
            cur.execute(draft_query, player['Player'])
            db_conn.commit()
            draftee = cur.fetchall()
            if not draftee:
                d = (999, 9999, 99)
            else:
                d = (draftee[0]['DraftPosition'], draftee[0]['Season'], draftee[0]['Team'])
            # get counts of draafted vs non-drafted as well
            if k == 'ChampionID':
                    final_teams['ChampionRoster'].append((player['Player'], d[0], d[1], d[2]))
                    if d[2] != final_teams['ChampionID']:
                        final_teams['ChampionCounts']['NonDrafted'] += 1
                    else:
                        final_teams['ChampionCounts']['Drafted'] += 1
            
            elif k == "RunnerUpID":
                final_teams['RunnerUpRoster'].append((player['Player'], d[0], d[1], d[2]))
                if d[2] != final_teams['RunnerUpID']:
                    final_teams['RunnerUpCounts']['NonDrafted'] += 1
                else:
                    final_teams['RunnerUpCounts']['Drafted'] += 1        

    finals.append(final_teams)

In [121]:
roster_counts = []
for final in finals:
    if final['Season'] >= 2000:
        print(final['Season'], final['ChampionCounts'], final['RunnerUpCounts'])
        roster_counts.append(final['ChampionCounts']['Drafted'] + final['ChampionCounts']['NonDrafted'])
        roster_counts.append(final['RunnerUpCounts']['Drafted'] + final['RunnerUpCounts']['NonDrafted'])

2017 {'Drafted': 12, 'NonDrafted': 19} {'Drafted': 14, 'NonDrafted': 17}
2016 {'Drafted': 13, 'NonDrafted': 14} {'Drafted': 13, 'NonDrafted': 13}
2015 {'Drafted': 14, 'NonDrafted': 15} {'Drafted': 13, 'NonDrafted': 14}
2014 {'Drafted': 13, 'NonDrafted': 9} {'Drafted': 8, 'NonDrafted': 16}
2013 {'Drafted': 13, 'NonDrafted': 15} {'Drafted': 8, 'NonDrafted': 18}
2012 {'Drafted': 11, 'NonDrafted': 14} {'Drafted': 13, 'NonDrafted': 15}
2011 {'Drafted': 8, 'NonDrafted': 15} {'Drafted': 12, 'NonDrafted': 23}
2010 {'Drafted': 12, 'NonDrafted': 12} {'Drafted': 9, 'NonDrafted': 17}
2009 {'Drafted': 13, 'NonDrafted': 17} {'Drafted': 13, 'NonDrafted': 13}
2008 {'Drafted': 15, 'NonDrafted': 13} {'Drafted': 13, 'NonDrafted': 14}
2007 {'Drafted': 3, 'NonDrafted': 19} {'Drafted': 12, 'NonDrafted': 11}
2006 {'Drafted': 5, 'NonDrafted': 17} {'Drafted': 12, 'NonDrafted': 13}
2004 {'Drafted': 6, 'NonDrafted': 16} {'Drafted': 6, 'NonDrafted': 17}
2003 {'Drafted': 10, 'NonDrafted': 15} {'Drafted': 7, 'NonDr

In [122]:
# total counts
print(sum(roster_counts) / len(roster_counts))

25.764705882352942


# Numbers

Let's get:  

The percentage since 2000 of drafted vs non-drafted on the two best rosters  
The number of teams with more drafted than non-drafted and vice-versa  


In [119]:
total_drafted = 0
total_ndrafted = 0
for final in finals:
    if final['Season'] >= 2000:
        total_drafted += final['ChampionCounts']['Drafted']
        total_ndrafted += final['ChampionCounts']['NonDrafted']
        total_drafted += final['RunnerUpCounts']['Drafted']
        total_ndrafted += final['RunnerUpCounts']['NonDrafted']
        
# average
print("ALL")
total = total_drafted+total_ndrafted
print("Total, Drafted, NonDrafted: {}, {}, {}".format(total, total_drafted, total_ndrafted))
print("Average Draft, Non: {} {}".format(total_drafted/total, total_ndrafted/total))

# average for champion
total_drafted = 0
total_ndrafted = 0
for final in finals:
    if final['Season'] >= 2000:
        total_drafted += final['ChampionCounts']['Drafted']
        total_ndrafted += final['ChampionCounts']['NonDrafted']
        
print("CHAMPION")
total = total_drafted+total_ndrafted
print("Total, Drafted, NonDrafted: {}, {}, {}".format(total, total_drafted, total_ndrafted))
print("Average Draft, Non: {} {}".format(total_drafted/total, total_ndrafted/total))

# average for runnerup
total_drafted = 0
total_ndrafted = 0
for final in finals:
    if final['Season'] >= 2000:
        total_drafted += final['RunnerUpCounts']['Drafted']
        total_ndrafted += final['RunnerUpCounts']['NonDrafted']
        
print("CHAMPION")
total = total_drafted+total_ndrafted
print("Total, Drafted, NonDrafted: {}, {}, {}".format(total, total_drafted, total_ndrafted))
print("Average Draft, Non: {} {}".format(total_drafted/total, total_ndrafted/total))
        

ALL
Total, Drafted, NonDrafted: 876, 352, 524
Average Draft, Non: 0.4018264840182648 0.5981735159817352
CHAMPION
Total, Drafted, NonDrafted: 426, 171, 255
Average Draft, Non: 0.4014084507042254 0.5985915492957746
CHAMPION
Total, Drafted, NonDrafted: 450, 181, 269
Average Draft, Non: 0.4022222222222222 0.5977777777777777


In [90]:
more_draft = 0
more_non = 0
same = 0

for final in finals:
    if final['Season'] >= 2000:
        if final['ChampionCounts']['Drafted'] > final['ChampionCounts']['NonDrafted']:
            more_draft += 1
        elif final['ChampionCounts']['NonDrafted'] > final['ChampionCounts']['Drafted']:
            more_non += 1
        elif final['ChampionCounts']['NonDrafted'] == final['ChampionCounts']['Drafted']:
            same += 1
        
        if final['RunnerUpCounts']['Drafted'] > final['RunnerUpCounts']['NonDrafted']:
            more_draft += 1
        elif final['RunnerUpCounts']['NonDrafted'] > final['RunnerUpCounts']['Drafted']:
            more_non += 1
        elif final['RunnerUpCounts']['NonDrafted'] == final['RunnerUpCounts']['Drafted']:
            same += 1
            
print(more_draft, more_non, same)

3 26 5


In [88]:
# average number of drafted players on a given top 2 roster
# number of rosters = 34
avg_draft = total_drafted / 34
avg_ndraft = total_ndrafted / 34
print(avg_draft)
print(avg_ndraft)

10.352941176470589
15.411764705882353


Most teams feature more non-drafted players than drafted. About 20% more actually. That said, we don't know the impact of drafted vs non-drafted players. We also need to check how many players each team drafts. I.e. leading up to your stanley cup run, how many players did you need to draft. I can image that in a decade window prior to a stanley cup win, a team is probably drafting somewhere in the range of 60-70 players. If they only have, on average, 10.35 drafted players on the roster, vs 15.4 non-drafted, would it make more sense to trade picks or assets for proven players?   

In [93]:
# we can reload the player tables, and join the data found within to the given rosters.
# important info is: age, and stats. GP and TOI can point to a player's utilization. 
# other stats to note are G/A/PTS, and plusminus

# load the data
allplayers = pd.read_excel("NHLPlayerData1967_2018.xlsx", sheet_name="Player Data")

In [95]:
allplayers['Player'] = allplayers[['First Name', 'Last Name']].agg(' '.join, axis=1)
allplayers.head()

,Last Name,First Name,Season,Age,Team,Pos,GP,G,A,PTS,...,Shot,TOI,OPS,DPS,PS,OGVT,DGVT,SGVT,GVT,Player
0,Aalto,Antti,1997-98,22.0,MDA,C,3,0,0,0,...,1,0.0,-0.1,0.0,-0.1,-0.2,-0.1,0.0,-0.2,Antti Aalto
1,Aalto,Antti,1998-99,23.0,MDA,C,73,3,5,8,...,61,685.0,-0.7,0.3,-0.3,-1.7,-0.5,0.0,-2.2,Antti Aalto
2,Aalto,Antti,1999-00,24.0,MDA,C,63,7,11,18,...,102,830.0,0.3,0.4,0.7,0.2,1.1,0.0,1.3,Antti Aalto
3,Aalto,Antti,2000-01,25.0,MDA,C,12,1,1,2,...,18,127.0,0.0,0.2,0.2,-0.2,0.4,0.0,0.2,Antti Aalto
4,Abbott,Spencer,2013-14,25.0,TOR,RW,1,0,0,0,...,2,5.0,0.0,-0.1,-0.1,-0.2,-0.2,0.0,-0.4,Spencer Abbott


In [111]:
# functions needed: pull player row from pandas table

# function --> fix dict that pandas is returning
def fix_pandas_dict(d):
    # where d is the dictionary
    new_d = {}
    for k, v in d.items():
        new_d[k] = v[0]
        
    return new_d

def get_player_row(player, season, df):
    season1 = "{}-{}".format(season-1, str(season)[-2:])

    row = df.loc[(df['Player'] == player) & (df['Season'] == season1)]
    row.reset_index(drop=True, inplace=True)
    
    return fix_pandas_dict(row.to_dict())



{'Last Name': 'Aalto', 'First Name': 'Antti', 'Season': '1999-00', 'Age': 24.0, 'Team': 'MDA', 'Pos': 'C', 'GP': 63, 'G': 7, 'A': 11, 'PTS': 18, 'P/M': -13, 'PIM': 26, 'ESG': 6, 'PPG': 1, 'SHG': 0, 'Shot': 102, 'TOI': 830.0, 'OPS': 0.3, 'DPS': 0.4, 'PS': 0.7, 'OGVT': 0.2, 'DGVT': 1.1, 'SGVT': 0.0, 'GVT': 1.3, 'Player': 'Antti Aalto'}


In [117]:
# we can get all the roster boys and their stats here
for final in finals:
    final['ChampionRosterStats'] = []
    final['RunnerUpRosterStats'] = []
    
    for player in final['ChampionRoster']:
        final['ChampionRosterStats'].append(get_player_row(player[0], final['Season'], allplayers))
        
    for player in final['RunnerUpRoster']:
        final['RunnerUpRosterStats'].append(get_player_row(player[0], final['Season'], allplayers))
    
